In [ ]:
import re
from matplotlib import matplotlib.pyplot as plt
from time import perf_counter
import json
import itertools
import ast
import pickle
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from pattern.en import pluralize, singularize
import enchant
import os
from deep_translator import (GoogleTranslator,
                             MicrosoftTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             PapagoTranslator,
                             DeeplTranslator,
                             QcriTranslator,
                             single_detection,
                             batch_detection)
from utils import word_cloud,get_sentences,spellcheck,lookup,count_None,show_missing,lookup_set,avg_flavor
#nltk.download('wordnet')


p=WordNetLemmatizer()
d = enchant.Dict("en_GB")





# Part 1
### this part contains
- extracting recipes from the text files
- text cleaning and removal of stopwords



In [ ]:
# extract the recipes alongside their names from each text file

ingridient_list = get_sentences('dataset Cuisine Mag\\abats.txt','COMPOSITION.','\d+\.')
recipe_names = get_sentences('dataset Cuisine Mag\\abats.txt','\d+\.','COMPOSITION')

# we repeat the same process for each seperate text file
for file in os.listdir('dataset Cuisine Mag'):
    if file.endswith('.txt'):
        print(file)

In [ ]:

data = {'recipe_name':recipe_names, 'Ingredients':ingridient_list}
data2=pd.read_csv('FlavorDB\\flavor_db.csv')
df = pd.DataFrame(data)

df['recipe_name']=df['recipe_name'].apply(lambda row:(row.split('\n')))
df['Ingredients']=df['Ingredients'].str.lower()
df['Ingredients']=df.Ingredients.str.replace(r"\((\s)?\w+(\s)?(\w+(\s)?)?\*\)", ' ',regex=True) # removes (kerwiya *)
df['Ingredients']=df.Ingredients.str.replace(r"tête d'(?=ail)", ' ',regex=True) #
df['Ingredients']=df.Ingredients.str.replace(r"\s*\(.*\)", '',regex=True)
df['Ingredients']=df.Ingredients.str.replace(r"bien mûres|[^-œàéèçâêôa-zA-Z0-9']|\bk?g (de)?\b|l'|louche|gousses?|pour friture|hachée?|[0-9] ou [0-9]|[0-9] à [0-9]", ' ',regex=True)
                                            #leaves french characters| removes kg and g| removes 4 ou 3 | removes 4 à 5

df['Ingredients']=df.Ingredients.str.replace(r"(?<!huile)d'(?!\bolive\b|youl|agneau|oiseaux)"," ",regex=True) #removes d' except for huile d'olive

df['Ingredients']=df.Ingredients.str.replace(r"(?<=sucre) semoule|glace|\s*cristal(l)?isé|(mondées et)? moulues|entier|fraiches?|fonds|assez|quarantaine de|concassée|\s*soup de|moulus|concassé|paquet de|mesure de|bien blanches|douzaine de|\bfleur de\b|viande\s*(?=agneau)|pieds? de?|trempées|bottes? (de)?|bâtonnet de|côtelettes de|peu de|\bviande de?\b|tête|feuilles? de|tranche de|durs?|décortiquées|remplie de|concentré de|pointe de|\s*pincées? de |variés|de préférence|\bune?\b|branches? de|rases? (de)?|facultatif|baguette de|quartier de|fort|rassis|tendres|sèches|poudre|râpé|sachet de|pulvérisé|bien remplie (de)?|fin(ement)?|bols? (de)?|litre (de)?|beaux|secs?|mises à tremper la veille|bottes? (de)?|feuilles? de (?!laurier)|\bà\s+huile\b|tête (?=ail)|rond|entières|(Q|q)uelques|gouttes de|boite de|soupçon de|tiède|\bun\b|\bà écosser\b|fraiches|tasse d?e?|gross?e?s?|petite?s?|moyenn?e?s?|bon(ne)?|belles|frais|pilée?|measure de|noix de|verres? (de|à eau|à thé)? (de)?|(petit)? pied de|[0-9]+ (de)?|cuillèree?s? à \w+ (de)?|cuillerée?s? (à \w+ (de|d)?|(de)?)|poignées? (de)?|bouquets? de|trempés\s?(ensemble)? la veille|environ (de)?|(en)? morceaux?|morceaux? de|livres? (de)?|un peu de|coupé|dés|\ben\b|rase ", ' ',regex=True)


df['Ingredients']=df['Ingredients'].str.lower() # lower case

df['Ingredients_alpha']=df['Ingredients'].apply(lambda row:row.split('-')) #split into a list of words

df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[w.lstrip(" ") for w in row] ) # removing spaces
df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[w.lstrip() for w in row] )
df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[w.rstrip() for w in row] )
df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[w for w in row if w !='']) #remove empty words
df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[re.sub(r"\bou\b.*",'',w) for w in row] ) # removing second part of OR
df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[re.sub(r"(?<=olives) .*",'',w) for w in row] ) # removing the color of olives
df['Ingredients_alpha']=df['Ingredients_alpha'].apply(lambda row:[w.strip() for w in row] )
df['Ingredients_alpha'] = df['Ingredients_alpha'].apply(lambda row : list(set(row)))
df.head()


In [ ]:
word_cloud(df['Ingredients_alpha'])

# Part 2
## this part contains
- replacing missing ingredients
- translation and spellchecking
- assigning each ingredient a set of molecules
- calculating average flavor sharing for both the savory and sweet dataset

In [ ]:


# we repeat the process for each pickle file ,you can just read all the pickle files and concatenate them into one big dataframe

data= pd.read_pickle(r"Pickle files/abats.pkl")


# list of words that exist but causes issues due to translation error
troubled_words={'coquillettes':'macaroni','courges':'butternut squash','couscous':'couscous',
                'écorces  orange':'orange oil','écorces de citron':'lemon peel oil','haricots blancs':'kidney beans',
                'cardes':'cardoon','smen':'butter','veau':'beef','chapelure':'bread','eau de fleur  oranger':'neroli oil',
                'mouton':'mutton','fèves':'beans','vermicelles':'pasta','cubèbe':'pepper','colorant':'saffron','sardines':'clupeinae',
                'pois':'peas','piment':'capsicum','piment rouge   et fort':'capsicum','poissons':'fish','poivre noir':'white pepper',
                'poivre rouge':'pepper','huile':'cooking oil','farine':'flour','betteraves':'beetroot','courgettes':'green zucchini',
               'pigeons':'squab','pois cassés':'peas','mie de pain':'bread','pruneaux':'plum','haricots blancs à  œil noir':'black-eyed pea'}


#replacing composed items with the list of ingredient they include
replace_items={'hror':['poivre','cannelle','clou de girofle','coriandre','noix de muscade','gingembre','cardamome'],
              'viande séchée':['bœuf','huile','ail','carvi','poivre rouge','poivre noir', 'coriandre','cumin','sel'],
              'dersa':['ail','cannelle','cumin','piment','sel','laurier'],
              'ras lhanout':['cumin','gingembre','sel','poivre','cannelle','coriandre','allspice','clove','piment'],
              'chermoula':['oignon','ail','persil','poivre rouge','cumin','poivre noir','citron','farine','sel','huile'],
              'crêpes':['semouline','farine','sel','eau'],
               'tlitli':['semouline','sel','eau'],
               "d'youl":['semouline','farine','sel','water'],
              'plombs':['semoule','semouline','sel'],
              'tous les légumes de la saison':['pomme de terre','carotte', 'navet', 'courgettes','poireau']}
#the replacement process
for i,ls in data.iterrows():
    expansions = (replace_items[e] if e.lower() in replace_items.keys() else [e] for e in ls['Ingredients_alpha'])
    expanded=[v for vals in expansions for v in vals]
    data.at[i,'Ingredients_alpha']=expanded




In [ ]:

#turning the list of ingredients into  dictionaries
data['Ingredients_alpha'] = data.Ingredients_alpha.apply(lambda x: ast.literal_eval(str(x)))
data['Ingredients_alpha'] = data.Ingredients_alpha.apply(lambda x:dict.fromkeys(x, ""))
for  element in data['Ingredients_alpha']:
    for k,v in element.items():
        #check each ingredient
        if k.strip() in troubled_words.keys():
            #if the word exists in troubled_words , replace it directly
            words=troubled_words[k.strip()]
        else:
            #else translate it and make it lower case
            translated = GoogleTranslator(source='fr', target='en').translate(text=k).lower()
            #spell check if the word exists in an english dictionary
            words=spellcheck(translated.split(' '),data2)
        element[k]=lookup(words,data2)




In [ ]:
# sanity check
count_None(data['Ingredients_alpha'])

## Average food sharing calculation

In [ ]:
# after repeating the process for all the clean pickle files , concatenate them into one dataframe
data_food=pd.read_csv('data_food_final.csv')

##create a dictionary of english ingredients
ls=[]
for element in data_food['eng_Ingredients_alpha']:
    element=ast.literal_eval(element)
    z=[]

    for k,v in (element).items():
        z.append(k)

    ls.append(z)

data_food['Ingredients_molecules']=ls
data_food['Ingredients_molecules']=data_food.Ingredients_molecules.apply(lambda x:dict.fromkeys(x, ""))

# find the molecular set for each ingredient
for element in data_food['Ingredients_molecules']:
    for k,v in element.items():
        element[k]=lookup_set(k,data2)

#calculate average flavor sharing
data_food['avg_flavor']=data_food['Ingredients_molecules'].apply(lambda x:avg_flavor(x))
final_score=sum(data_food['avg_flavor'])/data_food['avg_flavor'].shape[0]
print(final_score)





35.92777107479222


In [ ]:
#repeat the same process for sweet recipes
data_gat=pd.read_csv('data_gateaux.csv')


#drop this recipe because it only contains one ingredient
data_gat=data_gat.drop(79,axis=0)

# create a dictionary of english ingredients
ls=[]
for element in data_gat['eng_Ingredients_alpha']:
    element=ast.literal_eval(element)
    z=[]
    for k,v in element.items():
        z.append(k)

    ls.append(z)
data_gat['Ingredients_molecules']=ls
data_gat['Ingredients_molecules']=data_gat.Ingredients_molecules.apply(lambda x:dict.fromkeys(x, ""))


for element in data_gat['Ingredients_molecules']:
    for k,v in element.items():
        element[k]=lookup_set(k,data2)
data_gat['avg_flavor']=data_gat['Ingredients_molecules'].apply(lambda x:avg_flavor(x))
final_score=sum(data_gat['avg_flavor'])/data_gat['avg_flavor'].shape[0]
print(final_score)


14.469971139971143
